In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time



In [ ]:
class config:
    input = 700
    output = 20
    hid = 256    # RC Neurons
    
    batch = 256
    epoch = 50
    lr = 1e-3
    device = torch.device('cuda:0')
    
    

In [4]:
class ActFun(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.gt(0).float()

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        temp = abs(input - 0) < 0.5 # lens
        return grad_input * temp.float()

act_fun = ActFun.apply

In [ ]:
def mem_update(input, mem, spk, thr, alpha, rst):
    mem = rst * spk + mem * alpha * (1-spk) + input
    spike = act_fun(mem - thr)
    return mem, spike

class RC(nn.Module):
    def __init__(self) -> None:
        super(RC, self).__init__()
        input = config.input
        hid = config.hid
        self.fc_in = nn.Linear(input, hid)
        self.A = nn.Parameter(torch.rand(hid, hid), requires_grad=False) # 邻接矩阵
        
        
    def forward(self, input):
        batch, time_step, in_dim = input.shape
        hid_mem = torch.empty(batch, hid_dim).uniform_(0, 0.1)
        hid_spk = torch.zeros(batch, hid_dim)
        for t in range(time_step):
            